In [20]:
import random
import math
import gym
import numpy as np
import PIL
from PIL import Image
import matplotlib
import matplotlib.cm as cm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [21]:
class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.hidden_size=512
        self.rnn= nn.RNN(input_size=4, hidden_size=512,num_layers=2,batch_first=True)
        self.fc = nn.Linear(512, 2)
        
    def init_hidden(self,batch_size):
        return (torch.zeros(2,batch_size, self.hidden_size))
    
    def forward(self,x,hidden):
        print('hi',x)
        x=x.reshape(x.shape[0],1,4)
        x,h_0=self.rnn(x,hidden)
        return self.fc(x.contiguous().view(x.size(0), -1))
        

In [22]:
env = gym.make('CartPole-v0').unwrapped
env.reset()

array([ 0.03666071, -0.03619895, -0.03427681,  0.01783237])

In [23]:
policy=model()
target_net=model()
target_net.load_state_dict(policy.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy.parameters())
criterion = F.smooth_l1_loss
memory=10000
store=[[dict()] for i in range(memory)]
gamma=0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

In [24]:
def addEpisode(ind,prev,curr,reward,act):
    if len(store[ind]) ==0:
        store[ind][0]={'prev':prev,'curr':curr,'reward':reward,'action':act}
    else:
        store[ind].append({'prev':prev,'curr':curr,'reward':reward,'action':act})

In [25]:
def trainNet(total_episodes):
    if total_episodes==0:
        return
    ep=random.randint(0,total_episodes-1)
    if len(store[ep]) < 8:
        return
    else:  
        start=random.randint(1,len(store[ep])-1)
        length=len(store[ep])
        inp=[]
        target=[]
        rew=torch.Tensor(1,length-start)
        actions=torch.Tensor(1,length-start)
        
        for i in range(start,length,1):
            inp.append((store[ep][i]).get('prev'))
            target.append((store[ep][i]).get('curr'))
            rew[0][i-start]=store[ep][i].get('reward')
            actions[0][i-start]=store[ep][i].get('action')
        targets = torch.Tensor(target[0].shape[0],target[0].shape[1],target[0].shape[2])
        torch.cat(target, out=targets)
        ccs=torch.Tensor(inp[0].shape[0],inp[0].shape[1],inp[0].shape[2])
        torch.cat(inp, out=ccs)
        hidden = policy.init_hidden(length-start)
        qvals= target_net(targets,hidden)
        actions=actions.type('torch.LongTensor')
        actions=actions.reshape(length-start,1)
        hidden = policy.init_hidden(length-start)
        inps=policy(ccs,hidden).gather(1,actions)
        p1,p2=qvals.detach().max(1)
        targ = torch.Tensor(1,p1.shape[0])   
        for num in range(start,length,1):
            if num==len(store[ep])-1:
                targ[0][num-start]=rew[0][num-start] 
            else:
                targ[0][num-start]=rew[0][num-start]+gamma*p1[num-start]
        optimizer.zero_grad()
        inps=inps.reshape(1,length-start)
        loss = criterion(inps,targ)
        loss.backward()
        for param in policy.parameters():
            param.grad.data.clamp(-1,1)
        optimizer.step()
            

In [26]:
def trainDRQN(episodes):
    steps_done=0
    avg=0
    for i in range(0,episodes,1):
        if i%100==0 and i!=0:
            print("Episode",i,end=" ")
        env.reset()
        prev,_,_,_=env.step(0)
        prev=torch.from_numpy(prev)
        prev=prev.reshape(1,1,4)
        prev=prev.type('torch.FloatTensor')
        done=False
        steps=0
        rew=0
        while done == False:
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
            steps+=1
            hidden = policy.init_hidden(1)
            output=policy(prev,hidden)
            action=(output.argmax()).item()
            rand= random.uniform(0,1)
            if rand < eps_threshold:
                action=random.randint(0,1)
            x = env.step(action)
            #print(x)
            sc,reward,done,_=x
            avg=avg+reward
            sc=torch.from_numpy(sc)
            sc=sc.reshape(1,1,4)
            sc=sc.type('torch.FloatTensor')
            rew=rew+reward
            addEpisode(i,prev.unsqueeze(0),sc.unsqueeze(0),reward,action)
            trainNet(i)
            prev=sc
            steps_done+=1
        terminal = torch.zeros(prev.shape[0],prev.shape[1],prev.shape[2])
        addEpisode(i,prev.unsqueeze(0),terminal.unsqueeze(0),-10,action)
        if i%10==0:
            target_net.load_state_dict(policy.state_dict())
        if i%100==0 and i!=0:
            print(" -> Average reward over last 100 episodes " ,avg/100)
            avg=0

In [19]:
trainDRQN(2000)

hi tensor([[[-0.0379, -0.1588, -0.0415,  0.2912]]])
hi tensor([[[-0.0411, -0.3534, -0.0357,  0.5705]]])
hi tensor([[[-0.0482, -0.5480, -0.0243,  0.8517]]])
hi tensor([[[-0.0591, -0.3525, -0.0072,  0.5515]]])
hi tensor([[[-0.0662, -0.5475,  0.0038,  0.8419]]])
hi tensor([[[-0.0771, -0.3525,  0.0206,  0.5504]]])
hi tensor([[[-0.0842, -0.5479,  0.0317,  0.8495]]])
hi tensor([[[-0.0951, -0.7434,  0.0486,  1.1520]]])
hi tensor([[[-0.1100, -0.5490,  0.0717,  0.8749]]])
hi tensor([[[-0.1210, -0.7450,  0.0892,  1.1893]]])
hi tensor([[[-0.1359, -0.5511,  0.1130,  0.9258]]])
hi tensor([[[-0.1469, -0.7476,  0.1315,  1.2518]]])
hi tensor([[[-0.1619, -0.9441,  0.1565,  1.5826]]])
hi tensor([[[-0.1807, -0.7512,  0.1882,  1.3425]]])
hi tensor([[[-0.0035, -0.1808, -0.0352,  0.2822]]])
hi tensor([[[[-0.1469, -0.7476,  0.1315,  1.2518]]],


        [[[-0.1619, -0.9441,  0.1565,  1.5826]]],


        [[[-0.1807, -0.7512,  0.1882,  1.3425]]],


        [[[-0.1958, -0.9481,  0.2150,  1.6877]]],


        [

hi tensor([[[-0.1110, -0.4358,  0.1907,  0.7759]]])
hi tensor([[[[-0.1359, -0.5511,  0.1130,  0.9258]]],


        [[[-0.1469, -0.7476,  0.1315,  1.2518]]],


        [[[-0.1619, -0.9441,  0.1565,  1.5826]]],


        [[[-0.1807, -0.7512,  0.1882,  1.3425]]],


        [[[-0.1958, -0.9481,  0.2150,  1.6877]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[-0.1210, -0.7450,  0.0892,  1.1893]]],


        [[[-0.1359, -0.5511,  0.1130,  0.9258]]],


        [[[-0.1469, -0.7476,  0.1315,  1.2518]]],


        [[[-0.1619, -0.9441,  0.1565,  1.5826]]],


        [[[-0.1807, -0.7512,  0.1882,  1.3425]]],


        [[[-0.1958, -0.9481,  0.2150,  1.6877]]]])
hi tensor([[[-0.1198, -0.2438,  0.2062,  0.5487]]])
hi tensor([[[[-0.0591, -0.3525, -0.0072,  0.5515]]],


        [[[-0.0662, -0.5475,  0.0038,  0.8419]]],


        [[[-0.0771, -0.3525,  0.0206,  0.5504]]],


        [[[-0.0842, -0.5479,  0.0317,  0.8495]]],


        [[[-0.0951, -0.7434,  0.0486,  1.1520]]],


     

hi tensor([[[ 0.0055,  0.8008,  0.0068, -0.9337]]])
hi tensor([[[[-0.1141, -1.1588,  0.1313,  1.8094]]],


        [[[-0.1372, -0.9653,  0.1675,  1.5603]]],


        [[[-0.1565, -1.1620,  0.1987,  1.9002]]],


        [[[-0.1798, -1.3586,  0.2367,  2.2474]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[-0.0948, -0.9626,  0.1016,  1.4868]]],


        [[[-0.1141, -1.1588,  0.1313,  1.8094]]],


        [[[-0.1372, -0.9653,  0.1675,  1.5603]]],


        [[[-0.1565, -1.1620,  0.1987,  1.9002]]],


        [[[-0.1798, -1.3586,  0.2367,  2.2474]]]])
hi tensor([[[ 0.0215,  0.6056, -0.0118, -0.6388]]])
hi tensor([[[[-0.0771, -0.3525,  0.0206,  0.5504]]],


        [[[-0.0842, -0.5479,  0.0317,  0.8495]]],


        [[[-0.0951, -0.7434,  0.0486,  1.1520]]],


        [[[-0.1100, -0.5490,  0.0717,  0.8749]]],


        [[[-0.1210, -0.7450,  0.0892,  1.1893]]],


        [[[-0.1359, -0.5511,  0.1130,  0.9258]]],


        [[[-0.1469, -0.7476,  0.1315,  1.2518]]],


     

hi tensor([[[ 0.1143,  0.4179, -0.0863, -0.5128]]])
hi tensor([[[[-0.1100, -0.5490,  0.0717,  0.8749]]],


        [[[-0.1210, -0.7450,  0.0892,  1.1893]]],


        [[[-0.1359, -0.5511,  0.1130,  0.9258]]],


        [[[-0.1469, -0.7476,  0.1315,  1.2518]]],


        [[[-0.1619, -0.9441,  0.1565,  1.5826]]],


        [[[-0.1807, -0.7512,  0.1882,  1.3425]]],


        [[[-0.1958, -0.9481,  0.2150,  1.6877]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[-0.0951, -0.7434,  0.0486,  1.1520]]],


        [[[-0.1100, -0.5490,  0.0717,  0.8749]]],


        [[[-0.1210, -0.7450,  0.0892,  1.1893]]],


        [[[-0.1359, -0.5511,  0.1130,  0.9258]]],


        [[[-0.1469, -0.7476,  0.1315,  1.2518]]],


        [[[-0.1619, -0.9441,  0.1565,  1.5826]]],


        [[[-0.1807, -0.7512,  0.1882,  1.3425]]],


        [[[-0.1958, -0.9481,  0.2150,  1.6877]]]])
hi tensor([[[ 0.1226,  0.6142, -0.0966, -0.8314]]])
hi tensor([[[[-0.0591, -0.3525, -0.0072,  0.5515]]],


     

hi tensor([[[ 0.0549,  0.5345, -0.0467, -0.8113]]])
hi tensor([[[[ 0.,  0.,  0.,  0.]]]])
hi tensor([[[[-0.1798, -1.3586,  0.2367,  2.2474]]]])
hi tensor([[[ 0.0656,  0.3400, -0.0629, -0.5337]]])
hi tensor([[[[-0.0558,  0.0258,  0.0736,  0.1171]]],


        [[[-0.0553,  0.2198,  0.0759, -0.1514]]],


        [[[-0.0509,  0.4137,  0.0729, -0.4192]]],


        [[[-0.0426,  0.6077,  0.0645, -0.6881]]],


        [[[-0.0304,  0.8019,  0.0508, -0.9598]]],


        [[[-0.0144,  0.9963,  0.0316, -1.2361]]],


        [[[ 0.0055,  0.8008,  0.0068, -0.9337]]],


        [[[ 0.0215,  0.6056, -0.0118, -0.6388]]],


        [[[ 0.0337,  0.4106, -0.0246, -0.3499]]],


        [[[ 0.0419,  0.2159, -0.0316, -0.0651]]],


        [[[ 0.0462,  0.0212, -0.0329,  0.2175]]],


        [[[ 0.0466, -0.1734, -0.0286,  0.4996]]],


        [[[ 0.0431,  0.0221, -0.0186,  0.1980]]],


        [[[ 0.0436,  0.2175, -0.0146, -0.1004]]],


        [[[ 0.0479,  0.4128, -0.0166, -0.3977]]],


        [[[ 0.0562,  

...

hi tensor([[[-0.0319, -0.5934,  0.0034,  0.9017]]])
hi tensor([[[[ 0.0447, -0.3837, -0.0202,  0.5610]]],


        [[[ 0.0371, -0.5785, -0.0090,  0.8472]]],


        [[[ 0.0255, -0.3833,  0.0080,  0.5517]]],


        [[[ 0.0178, -0.5785,  0.0190,  0.8469]]],


        [[[ 0.0062, -0.3837,  0.0359,  0.5603]]],


        [[[-0.0014, -0.5793,  0.0471,  0.8640]]],


        [[[-0.0130, -0.7750,  0.0644,  1.1712]]],


        [[[-0.0285, -0.9709,  0.0878,  1.4833]]],


        [[[-0.0479, -0.7770,  0.1175,  1.2193]]],


        [[[-0.0635, -0.5835,  0.1419,  0.9656]]],


        [[[-0.0751, -0.7802,  0.1612,  1.2993]]],


        [[[-0.0907, -0.5875,  0.1872,  1.0611]]],


        [[[-0.1025, -0.7845,  0.2084,  1.4062]]],


        [[[-0.1182, -0.5925,  0.2365,  1.1853]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[ 0.0485, -0.1890, -0.0257,  0.2765]]],


        [[[ 0.0447, -0.3837, -0.0202,  0.5610]]],


        [[[ 0.0371, -0.5785, -0.0090,  0.8472]]],


       

hi tensor([[[-0.0310, -0.3618,  0.0879,  0.7044]]])
hi tensor([[[[ 0.0122, -0.4267,  0.0107,  0.5616]]],


        [[[ 0.0037, -0.6220,  0.0219,  0.8576]]],


        [[[-0.0088, -0.8174,  0.0390,  1.1571]]],


        [[[-0.0251, -1.0130,  0.0622,  1.4617]]],


        [[[-0.0454, -0.8187,  0.0914,  1.1891]]],


        [[[-0.0617, -1.0149,  0.1152,  1.5090]]],


        [[[-0.0820, -0.8213,  0.1454,  1.2544]]],


        [[[-0.0985, -0.6283,  0.1705,  1.0105]]],


        [[[-0.1110, -0.4358,  0.1907,  0.7759]]],


        [[[-0.1198, -0.2438,  0.2062,  0.5487]]],


        [[[-0.1246, -0.4411,  0.2172,  0.8986]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[ 0.0246, -0.6219, -0.0065,  0.8563]]],


        [[[ 0.0122, -0.4267,  0.0107,  0.5616]]],


        [[[ 0.0037, -0.6220,  0.0219,  0.8576]]],


        [[[-0.0088, -0.8174,  0.0390,  1.1571]]],


        [[[-0.0251, -1.0130,  0.0622,  1.4617]]],


        [[[-0.0454, -0.8187,  0.0914,  1.1891]]],


       

hi tensor([[[[-0.0453, -0.0080,  0.0178,  0.0226]]],


        [[[-0.0455,  0.1869,  0.0182, -0.2644]]],


        [[[-0.0417,  0.3817,  0.0130, -0.5513]]],


        [[[-0.0341,  0.1864,  0.0019, -0.2545]]],


        [[[-0.0304, -0.0087, -0.0032,  0.0388]]],


        [[[-0.0305, -0.2038, -0.0024,  0.3305]]],


        [[[-0.0346, -0.0087,  0.0042,  0.0370]]],


        [[[-0.0348,  0.1864,  0.0050, -0.2543]]],


        [[[-0.0310,  0.3814, -0.0001, -0.5454]]],


        [[[-0.0234,  0.1863, -0.0110, -0.2528]]],


        [[[-0.0197, -0.0086, -0.0161,  0.0364]]],


        [[[-0.0199, -0.2035, -0.0154,  0.3240]]],


        [[[-0.0239, -0.3984, -0.0089,  0.6118]]],


        [[[-0.0319, -0.5934,  0.0034,  0.9017]]],


        [[[-0.0438, -0.3983,  0.0214,  0.6100]]],


        [[[-0.0517, -0.5938,  0.0336,  0.9094]]],


        [[[-0.0636, -0.7893,  0.0518,  1.2124]]],


        [[[-0.0794, -0.9851,  0.0760,  1.5209]]],


        [[[-0.0991, -1.1810,  0.1064,  1.8363]]],


        [

hi tensor([[[-0.0347, -0.1542, -0.0302,  0.3262]]])
hi tensor([[[[-0.0346, -0.0087,  0.0042,  0.0370]]],


        [[[-0.0348,  0.1864,  0.0050, -0.2543]]],


        [[[-0.0310,  0.3814, -0.0001, -0.5454]]],


        [[[-0.0234,  0.1863, -0.0110, -0.2528]]],


        [[[-0.0197, -0.0086, -0.0161,  0.0364]]],


        [[[-0.0199, -0.2035, -0.0154,  0.3240]]],


        [[[-0.0239, -0.3984, -0.0089,  0.6118]]],


        [[[-0.0319, -0.5934,  0.0034,  0.9017]]],


        [[[-0.0438, -0.3983,  0.0214,  0.6100]]],


        [[[-0.0517, -0.5938,  0.0336,  0.9094]]],


        [[[-0.0636, -0.7893,  0.0518,  1.2124]]],


        [[[-0.0794, -0.9851,  0.0760,  1.5209]]],


        [[[-0.0991, -1.1810,  0.1064,  1.8363]]],


        [[[-0.1227, -1.3771,  0.1432,  2.1600]]],


        [[[-0.1503, -1.5734,  0.1864,  2.4933]]],


        [[[-0.1817, -1.7695,  0.2362,  2.8369]]],


        [[[ 0.0000,  0.0000,  0.0000,  0.0000]]]])
hi tensor([[[[-0.0305, -0.2038, -0.0024,  0.3305]]],


       

KeyboardInterrupt: 